In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
from gensim.models import FastText

In [ ]:
!nvidia-smi

In [ ]:
df = pd.read_csv("/kaggle/input/commonlitreadabilityprize/train.csv")

In [ ]:
df.head()

In [ ]:
df = df.drop(["standard_error"],axis=1)

In [ ]:
df

In [ ]:
import random

In [ ]:
import tensorflow as tf

In [ ]:
ind = random.randint(1,2834-5)

df.iloc[ind].target,df.iloc[ind].excerpt

In [ ]:
df.describe()

In [ ]:
def clean_text(text):
    text = re.sub('\n', '', text)
    return text

In [ ]:
import re
import string

In [ ]:
df["excerpt"] = df["excerpt"].apply(lambda text: clean_text(text))

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [ ]:
df["length"] = df["excerpt"].apply(lambda text: len(text.split()))

In [ ]:
df.describe()

In [ ]:
max_length = 210

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df["excerpt"],df["target"],test_size=0.2,random_state=42)

In [ ]:
import tensorflow_hub as hub

embed = hub.load("/kaggle/input/uselarge/")

In [ ]:
embedding = hub.KerasLayer(embed,trainable=False)

### embedding = tf.keras.layers.Embedding(input_dim=30000,output_dim=128,input_length=max_length)

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
inputs = layers.Input(shape=[],dtype=tf.string)
x = embedding(inputs)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128,activation="relu")(x)
x = layers.Dense(128,activation="relu")(x)
outputs = layers.Dense(1)(x)

In [ ]:
model_6 = tf.keras.Model(inputs,outputs)

In [ ]:
model_6.summary()

In [ ]:
model_6.compile(loss=tf.keras.losses.mse,optimizer=tf.keras.optimizers.Adam(),metrics=tf.keras.metrics.RootMeanSquaredError())

In [ ]:
import datetime

In [ ]:
def create_tensorboard_callback(dir_name,experiment_name):
  """
  Creates a TensorBoard callback instand to store log files.
  Stores log files with the filepath:
    "dir_name/experiment_name/current_datetime/"
  Args:
    dir_name: target directory to store TensorBoard log files
    experiment_name: name of experiment directory (e.g. efficientnet_model_1)
  """
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath="checkpoint/check.ckpt",monitor="val_root_mean_squared_error",save_best_only=True,save_weights_only=True)

In [ ]:
model_6.fit(X_train,y_train,epochs=5,validation_data=(X_test,y_test),callbacks=[create_tensorboard_callback("CommonLit","USE-Dense-FineTuneAll"),model_checkpoint])

In [ ]:
 model_7 = model_6

In [ ]:
embedding.trainable=True
for layer in embedding.layers[:-10]:
    layer.trainable = False

In [ ]:
embedding.layers

In [ ]:
ind = random.randint(1,2834-5)
df.iloc[ind:ind+5]

In [ ]:
df.iloc[0].excerpt

In [ ]:
df_test = pd.read_csv("/kaggle/input/commonlitreadabilityprize/test.csv")

In [ ]:
df_test

In [ ]:
preds = model_6.predict(df_test["excerpt"])

In [ ]:
preds

In [ ]:
submissions = pd.read_csv("/kaggle/input/commonlitreadabilityprize/sample_submission.csv")

In [ ]:
submissions

In [ ]:
submissions["target"] = preds

In [ ]:
submissions

In [ ]:
submissions.to_csv("/kaggle/working/submission.csv",index=False)